# Projet Final Apache Spark

**Nom Etudiant : DIEYE**  

**Prenom Etudiant: Babacar Sedikhe**  

**Classe : MIA**  


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [1]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.52.128:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1610975616128)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [2]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [3]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [5]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [6]:
fireDF.cache()

res0: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [7]:
fireDF.count()

res1: Long = 175296


In [8]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [9]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [10]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [19]:
// Reponse 1
/*Elimination des valeurs "nulles" puis selection et affichage du nombre
    d'appels distincts passés*/
val callTypeDistinct = fireDF.na.drop(cols = Seq("CallType")).
    select(countDistinct("CallType")).
    show

+------------------------+
|count(DISTINCT CallType)|
+------------------------+
|                      30|
+------------------------+



callTypeDistinct: Unit = ()


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [32]:
// Reponse 2
val differentCallType = fireDF.na.drop(cols = Seq("CallType")).
    groupBy("CallType").
    count.
    show

+--------------------+-----+
|            CallType|count|
+--------------------+-----+
|Elevator / Escala...|  453|
|         Marine Fire|   14|
|  Aircraft Emergency|   36|
|Confined Space / ...|   13|
|      Administrative|    3|
|              Alarms|19406|
|Odor (Strange / U...|  490|
|Citizen Assist / ...| 2524|
|              HazMat|  124|
|Watercraft in Dis...|   28|
|           Explosion|   89|
|           Oil Spill|   21|
|        Vehicle Fire|  854|
|  Suspicious Package|   15|
|Extrication / Ent...|   28|
|               Other| 2166|
|        Outside Fire| 2094|
|   Traffic Collision| 7013|
|       Assist Police|   35|
|Gas Leak (Natural...|  764|
+--------------------+-----+
only showing top 20 rows



differentCallType: Unit = ()


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [44]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
//Filtrage des réponses suppérieurs à 5 minutes
newFireDF.select("CallNumber","ResponseDelayedinMins").filter("ResponseDelayedinMins>5").show()
    

+----------+---------------------+
|CallNumber|ResponseDelayedinMins|
+----------+---------------------+
|  20110315|                 5.35|
|  20120147|                 6.25|
|  20130013|                  5.2|
|  20140067|                  5.6|
|  20140177|                 7.25|
|  20150056|            11.916667|
|  20150254|             5.116667|
|  20150265|             8.633333|
|  20150265|             95.28333|
|  20150380|                 5.45|
|  20150414|                  7.6|
|  20160059|             6.133333|
|  20160064|            5.1833334|
|  20170118|            6.9166665|
|  20170342|                  5.2|
|  20180129|                 6.35|
|  20180191|             7.983333|
|  20180382|                13.55|
|  20190062|                 5.15|
|  20190097|            13.583333|
+----------+---------------------+
only showing top 20 rows



newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


### Transformations des dates
Maintenant nous allons d'abord:
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard
2. Retourner le Dataframe transformée
3. Mettre en cache le nouveau DataFrame

In [45]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res32: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [61]:
//Reponse 4
fireTSDF.groupBy("CallType").count().
    sort(desc("count")).show()

+--------------------+------+
|            CallType| count|
+--------------------+------+
|    Medical Incident|113794|
|      Structure Fire| 23319|
|              Alarms| 19406|
|   Traffic Collision|  7013|
|Citizen Assist / ...|  2524|
|               Other|  2166|
|        Outside Fire|  2094|
|        Vehicle Fire|   854|
|Gas Leak (Natural...|   764|
|        Water Rescue|   755|
|Odor (Strange / U...|   490|
|   Electrical Hazard|   482|
|Elevator / Escala...|   453|
|Smoke Investigati...|   391|
|          Fuel Spill|   193|
|              HazMat|   124|
|Industrial Accidents|    94|
|           Explosion|    89|
|Train / Rail Inci...|    57|
|  Aircraft Emergency|    36|
+--------------------+------+
only showing top 20 rows



### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [63]:
//Reponse 5-a
val postBox = fireTSDF.groupBy("zipcode").count().
    sort(desc("count")).show

+-------+-----+
|zipcode|count|
+-------+-----+
|  94102|21840|
|  94103|20897|
|  94110|14801|
|  94109|14686|
|  94124| 9236|
|  94112| 8421|
|  94115| 7812|
|  94107| 6941|
|  94122| 6355|
|  94133| 6246|
|  94117| 5804|
|  94114| 5175|
|  94118| 5157|
|  94134| 5009|
|  94121| 4555|
|  94132| 4321|
|  94105| 4236|
|  94108| 4084|
|  94116| 3933|
|  94123| 3719|
+-------+-----+
only showing top 20 rows



postBox: Unit = ()


### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [71]:
//Reponse 5-b
val street = fireTSDF
    .select("Address", "Zipcode") 
    .where($"Zipcode" =!= 94102 && $"Zipcode" =!= 94103)
street.show(false)

+------------------------------+-------+
|Address                       |Zipcode|
+------------------------------+-------+
|2000 Block of CALIFORNIA ST   |94109  |
|0 Block of SILVERVIEW DR      |94124  |
|APPLETON AV/MISSION ST        |94110  |
|1400 Block of SUTTER ST       |94109  |
|BEALE ST/FOLSOM ST            |94105  |
|0 Block of FARALLONES ST      |94112  |
|1500 Block of WEBSTER ST      |94115  |
|DIAMOND ST/MARKET ST          |94114  |
|2700 Block of MISSION ST      |94110  |
|BRUNSWICK ST/GUTTENBERG ST    |94112  |
|1000 Block of SUTTER ST       |94109  |
|100 Block of 21ST AVE         |94121  |
|700 Block of CAPP ST          |94110  |
|700 Block of CAPP ST          |94110  |
|700 Block of CAPP ST          |94110  |
|200 Block of LAGUNA HONDA BLVD|94116  |
|BALBOA ST/PARK PRESIDIO BL    |94118  |
|BALBOA ST/PARK PRESIDIO BL    |94118  |
|2300 Block of MARKET ST       |94114  |
|500 Block of BROADWAY         |94133  |
+------------------------------+-------+
only showing top

street: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Address: string, Zipcode: int]


### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [87]:
//Reponse 6
//Nombre total d'appels
fireTSDF.select(sum("CallType")).show

//Moyenne des appels
fireTSDF.select(mean("ResponseDelayedinMins")).show

//Minimum des appels
fireTSDF.select(min("ResponseDelayedinMins")).show

//Maximum des appels
fireTSDF.select(max("ResponseDelayedinMins")).show

+-------------+
|sum(CallType)|
+-------------+
|         null|
+-------------+

+--------------------------+
|avg(ResponseDelayedinMins)|
+--------------------------+
|         3.892364154521585|
+--------------------------+

+--------------------------+
|min(ResponseDelayedinMins)|
+--------------------------+
|               0.016666668|
+--------------------------+

+--------------------------+
|max(ResponseDelayedinMins)|
+--------------------------+
|                   1844.55|
+--------------------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [88]:
//Reponse 7-a
fireTSDF.select(countDistinct(year(fireTSDF("AvailableDtTS")))).show

+-------------------------------------------------+
|count(DISTINCT year(CAST(AvailableDtTS AS DATE)))|
+-------------------------------------------------+
|                                               19|
+-------------------------------------------------+



### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [19]:
//Reponse 7-b
/*
Ecrire ici votre code
*/

### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [20]:
//Reponse 8
/*
Ecrire ici votre code
*/

### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [97]:
//Reponse 9
fireTSDF.write.parquet("datasets/sf-fire/sf-fire-calls.schema.parquet")

### Question 10
**Comment relire les données stockée en format Parquet?**

In [96]:
//Reponse 10
val newsfireTSDF = spark.read.parquet("datasets/sf-fire/sf-fire-calls.schema.parquet/")
newsfireTSDF.printSchema()

org.apache.spark.sql.AnalysisException:  Path does not exist: file:/home/geek95/Programmation fonctionnelle/datasets/sf-fire/sf-fire-calls.schema.parquet;

## FIN